<img src="../Images/DSC_Logo.png" style="width: 400px;">

# Time Series Theory in Python - Part 1: Exploratory Time Series Analysis

This notebook focuses on introducing basic concepts of time series analysis for univariate time series and then extending to some specific analysis for multivariate time series. 

In [ ]:
pip install PythonTsa statsmodels openpyxl seaborn scikit-learn

In [ ]:
from PythonTsa.datadir import getdtapath # We use the Python package PythonTsa to load datasets coming with this package.
dtapath=getdtapath()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. Stationarity

In [ ]:
# Generate synthetic time series data
time = pd.date_range(start="2020-01-01", periods=100)
trend = np.arange(100)  # Linear trend as a NumPy array
seasonal = [10 * np.sin(i / 5) for i in range(100)]  # Seasonal component
noise = np.random.normal(0, 2, size=100)  # Random noise

# Combine trend, seasonal, and noise components into one array
data = trend + seasonal + noise

# Create DataFrame
series = pd.Series(data, index=time)

# Plot the time series
series.plot()
plt.title("Synthetic time series with trend and seasonality")
plt.xlabel("Date")
plt.ylabel("Value")
plt.show()

## 2. Autocorrelation

In [ ]:
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pandas.plotting import lag_plot

### Autocorrelation Function (ACF):

The ACF Plot enables the evaluation of the correlation between an observation and its previous values.

In [ ]:
plot_acf(series, lags=30, alpha=0.05) # alpha=0.05 is the default)
plt.title("ACF of the Time Series")
plt.show()

### Partial Autocorrelation Function (PACF):

The PACF Plot helps to understand the direct relationship between an observation and its own lagged values, excluding the effects of other lags. The PACF clearly measures the correlation between $X_t$ and $X_{t-k}$ that is not explained by $\{ X_{t-k+1}, \dots, X_{t-1} \}$.

In [ ]:
plot_pacf(series, lags=30)
plt.title("PACF of the Time Series")
plt.show()

### Lag Plot:

The Lag Plot offers a scatter plot that directly compares the values of the time series to its lagged versions, helping to identify any linear relationships.

In [ ]:
lag_plot(series)
plt.title('Lag Plot of the Time Series')
plt.tight_layout()
plt.show()

### **Example 1: Chinese Quarterly GDP**

In [ ]:
# Load the data
x = pd.read_csv(dtapath + 'gdpquarterlychina1992.1-2017.4.csv',header=0)
dates = pd.date_range(start='1992',periods=len(x),freq='QE')
x.index=dates

# Plot the time series
x.plot()
plt.title('Chinese Quarterly GDP 1992-2017')
plt.ylabel('billions of RMB')
plt.show()

# Plot ACF
plot_acf(x, lags=60)
plt.title("ACF of the Time Series")
plt.show()

# Plot Lag Plot
lag_plot(x)
plt.title('Lag Plot of the Time Series')
plt.tight_layout()
plt.show()

### **Example 2: Quarterly Exchange Rates of GBP to NZ Dollar**

In [ ]:
# Load the exchange rate data
x = pd.read_csv(dtapath + 'ExchRate NZ per UK.txt', header=0)

# Create a date range starting from 1991 with quarterly frequency
dates = pd.date_range('1991', periods=len(x), freq='QE')

# Set the index to the created date range
x.index = dates

# Create a time series from the 'xrate' column
xts = pd.Series(x['xrate'])

# Plot the time series
xts.plot()
plt.xlabel('Years')
plt.ylabel('Exchange rate')
plt.title('Exchange Rate NZ per UK (1991 onward)')
plt.show()

# Plot ACF
fig = plt.figure()
plot_acf(xts, lags=17)
plt.title("ACF of the Time Series")
plt.show()

# Plot PACF
fig = plt.figure()
plot_pacf(xts, lags=17)
plt.title("PACF of the Time Series")
plt.show()

# Plot Lag Plot
lag_plot(xts)
plt.title('Lag Plot of the Time Series')
plt.tight_layout()
plt.show()

## 3. Random walks

Three simulated paths (time plots) of the standard normal random walk:

In [ ]:
from numpy.random import normal

In [ ]:
np.random.seed(1357)
a=normal(size=300); b=normal(size=300); c=normal(size=300)
x=np.cumsum(a); y=np.cumsum(b); z=np.cumsum(c)
xyz=pd.DataFrame({'x': x, 'y': y, 'z': z})
xyz.index=range(1,301)
xyz.plot(style=['-', '--', ':']); plt.show() # style means matplotlib line style per column

## 4. White noise

## 4.1 Simulating a Gaussian white noise

In [ ]:
from numpy import random

In [ ]:
random.seed(135) # for repeat
x=random.normal(loc=0, scale=1, size=1000)
xts=pd.Series(x)
xts.plot(); plt.xlabel('Time')
plt.ylabel('Simulated white noise')
plt.show()

plot_acf(xts, lags=30) # plotting ACF
plt.title("ACF of the Time Series")
plt.show()

It is an intuitive method for testing that a stationary time series is a white noise or not: To examine its ACF plot and if the ACF plot is similar to this figure, then we are apt to think that the time series is a white noise.

## 4.2 Chaos like a white noise

In [ ]:
# Initialize an empty pandas Series with a float data type
x = pd.Series(dtype=float)

# Start value for y
y = 0.3

# Use a for loop to generate the values
for t in range(1, 501):
    y = 4.0 * y * (1 - y)  # Logistic map equation
    x.loc[t-1] = y  # Assigning the value at index t-1

# Set the index of the series to be in the range 1 to 500
x.index = range(1, 501)

# Display the result
print(x)

In [ ]:
x.plot()
plt.xlabel('Time')
plt.ylabel('Simulated Chaos')
plt.show()

plot_acf(x, lags=30) # plotting ACF
plt.title("ACF of the Time Series")
plt.show()

## 4.3 White noise test

How to statistically test whether a stationary time series is a white noise?

For a stationary time series $\{X_t\}$ is a white noise if and only if its autocorrelation function $(ACF) \rho_k = 0$ for any integer $k = 0$.

### **Example 2 [continues]: Quarterly Exchange Rates of GBP to NZ Dollar**

The quarterly exchange rates of GBP to NZD are a financial time series. We often analyze their logarithms because this stabilizes the average and variability, reduces trends, and makes the data more normal. This is important due to the volatility in financial data. Logging also highlights relative changes instead of absolute values, which is key for understanding percentage returns and growth rates in finance.

In [ ]:
# Logarithm of the series:
logxts = np.log(xts)

We need to difference the time series to achieve stationarity before testing for white noise. Differencing will be more closely investigated later.

In [ ]:
dlogxts = logxts.diff(1).dropna()  # Remove NaN values resulting from differencing

Test for white noise:

In [ ]:
# Plotting the differenced logarithmic series
dlogxts.plot(marker='o', markersize=5)
plt.title('Difference of Logarithm of the ExchRate NZ per UK')
plt.xlabel('Time')
plt.ylabel('Difference of Log Price')
plt.show()

# Plotting the ACF
plot_acf(dlogxts, lags=17)
plt.title("ACF of the Time Series")
plt.show()

# Calculating ACF, Ljung-Box statistics, and p-values
r, q, p = acf(dlogxts, nlags=35, qstat=True)
print('ACF values:', r) 
print('Ljung-Box statistics:', q)
print('P-values:', p)

The Ljung-Box statistic checks if there are significant autocorrelations at multiple lags. The null hypothesis is that the data are independently distributed (i.e., white noise). If the p-value is low (commonly less than 0.05), we reject the null hypothesis.

The differentiated time series, indicated by the p-values from the Ljung-Box test, suggests that it behaves like white noise at higher lags since the p-values for those lags are greater than 0.05, indicating no significant autocorrelation. However, since the earlier p-values (for the first few lags) indicate some level of autocorrelation, we can't definitively say that the entire differentiated time series is purely white noise. It is approaching white noise behavior but might still have some dependencies at the initial lags.

## 5. Time Series Features: Time Series Decomposition

 In practice, many of realistic time series possess either deterministic seasonality (component) or a deterministic trend (component). Some of them may have both. After extracting the trend and seasonal components from a time series, the remainder is its random (variation) component. Decomposing a time series is helpful to better understand it and improve forecast accuracy.

In [ ]:
from statsmodels.graphics.tsaplots import month_plot
from statsmodels.tsa.seasonal import seasonal_decompose

### **Example 3: Australian Employed Total Persons**

The "Australia Employed Total Persons" dataset tracks the monthly total number of employed people in Australia from February 1978 to November 2018. Initially, the time series shows a steady upward trend with no apparent seasonality. However, when zooming in on data from January 2013 to January 2017, clear seasonal patterns emerge, repeating every year. Seasonal plots confirm that these patterns are consistent with little variation over time.

In [ ]:
# Get the data file path
dtapath = getdtapath()

# Load the Excel file containing employment data
aul = pd.read_excel(dtapath + 'AustraliaEmployedTotalPersons.xlsx', header=0)

# Create a time index starting from February 1978 with monthly frequency
timeindex = pd.date_range('1978-02', periods=len(aul), freq='ME')
aul.index = timeindex

# Extract the 'EmployedP' column for analysis
aults = aul['EmployedP']

# Plot the time series
aults.plot()
plt.title('Total Employed Persons in Australia')
plt.ylabel('Number of Persons Employed')
plt.xlabel('Date')
plt.show()

# Graph time series plot from January 2013 to January 2017
aults['2013-01':'2017-01'].plot()
plt.title('Total Employed Persons (2013-01 to 2017-01)')
plt.ylabel('Number of Persons Employed')
plt.xlabel('Date')
plt.show()

# Plot seasonal plots
month_plot(aults)
plt.title('Monthly Employment Data Seasonal Plot')
plt.show()

Using an additive model to decompose the time series:

In [ ]:
# Decompose the time series using an additive model (Time series=Trend+Seasonality+Residual)
aultsdeca = seasonal_decompose(aults, model='additive')
aultsdeca.plot()
plt.title('Seasonal Decomposition of Employment Data')
plt.show()

After decomposition, the residuals should ideally represent the random noise that remains after removing trend and seasonal patterns. By plotting the ACF of the residuals, we can determine if any patterns or correlations were not captured by the decomposition. A random noise pattern should show insignificant autocorrelation (i.e., ACF values near zero). 
In addition, rolling means help evaluate if the average of the residuals is constant, indicating stationarity. A trend in the rolling mean suggests a non-stationary component. Rolling standard deviations reveal the variability of the residuals; significant changes may indicate heteroscedasticity.

In [ ]:
# Extract residuals from the decomposition and drop NaN values
aultsdecaResid = aultsdeca.resid.dropna()

# Plot ACF of the residuals
plot_acf(aultsdecaResid, lags=48)
plt.title('ACF of Residuals')
plt.show()

# Rolling mean and standard deviation
rolm = aultsdecaResid.rolling(window=36, center=True).mean()
rolstd = aultsdecaResid.rolling(window=36, center=True).std()

# Plotting residuals, rolling mean, and rolling std
plt.plot(aultsdecaResid, label='Decomposed Residuals')
plt.plot(rolm, label='Rolling Mean', linestyle='--')
plt.plot(rolstd, label='Rolling Std', linestyle=':', c='red')
plt.title('Residuals of Australian Employed Persons')
plt.legend()
plt.show()

The residuals are mostly stationary, though there are seasonal correlations at specific lags.

## 6. Multivariate Time Series

In multivariate time series analysis, we examine multiple interrelated variables simultaneously instead of just one. This analysis helps us understand how these variables influence each other over time and enables joint forecasting. The methods used for analyzing single time series can often be adapted for multivariate scenarios, but it becomes essential to consider how the interactions between these variables affect their individual behaviors. The following are some specific analyses for exploring the dynamic relationships and dependencies inherent in multivariate datasets and time series.

### **Example 4: Large Dataset of Synthetic Time Series**

We generate multiple synthetic independent time series with trend, seasonal effect and random noise.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic time series data
dates = pd.date_range('2000-01-01', periods=100, freq='M')

# Create a DataFrame to hold the time series
data = pd.DataFrame(index=dates)

# Generate time series
n_series = 24
for i in range(1, n_series + 1):  # 24 time series
    trend = np.linspace(0, np.random.uniform(0.1, 1), len(dates))  # Linear trend
    seasonal = 0.1 * np.sin(np.linspace(0, 3 * np.pi, len(dates)))  # Seasonal effect
    noise = np.random.normal(loc=0, scale=0.05, size=len(dates))  # Random noise
    series = trend + seasonal + noise  # Combine to create the time series
    data[f'Series_{i}'] = series

# Plotting all the time series; highlight the last two series
plt.figure()
for column in data.columns:
    plt.plot(data.index, data[column], label=column, color='blue')
plt.title('Synthetic Time Series Data')
plt.xlabel('Date')
plt.ylabel('Values')
plt.grid()
plt.tight_layout()  # Adjust the layout
plt.show()

## 6.1 Pearson Correlation

Pearson correlation measures how strong and in what direction two continuous variables are related. It ranges from -1 to 1: a value of 1 means a perfect positive relationship, -1 means a perfect negative relationship, and 0 means no relationship. It looks at how much one variable changes in relation to another and is commonly used to understand connections between variables.

In [ ]:
# calculate Pearson correaltion
correlation_matrix = data.corr()

# plot heatmap
import seaborn as sns # another library for plotting
plt.figure()
sns.heatmap(correlation_matrix, annot=False, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.title('Pearson Correlation Heatmap')
plt.show()

# Find the two most correlated series
corr_values = correlation_matrix.unstack().sort_values(ascending=False)
corr_values = corr_values[corr_values < 1.0] # Exclude self-correlations (1.0) 
most_correlated_pair = corr_values.idxmax()
series1, series2 = most_correlated_pair
print(f'Most Correlated Series: {series1} and {series2}')

# Plotting all the time series, highlighting the two most correlated series
plt.figure()
for column in data.columns:
    plt.plot(data.index, data[column], label=column, color='lightgrey')
plt.plot(data.index, data[series1], label=series1, color='blue', linewidth=2.5)  # First highlighted series
plt.plot(data.index, data[series2], label=series2, color='blue', linewidth=2.5)  # Second highlighted series
plt.title('Synthetic Time Series Data')
plt.xlabel('Date')
plt.ylabel('Values')
plt.grid()
plt.tight_layout()
plt.show()

## 6.2 Cross-Correlation Analysis

The cross-correlation between the two selected time series measures how similar they are at different time lags. We limit the analysis to a maximum of 15 lags.

In [ ]:
from statsmodels.tsa.stattools import ccf

max_lag = 15
cross_corr = ccf(data[series1], data[series2])[:max_lag]

# Plot Cross-Correlation
plt.figure()
plt.stem(range(max_lag), cross_corr)
plt.title(f"Cross-Correlation between {series1} and {series2}")
plt.xlabel("Lags")
plt.ylabel("Cross-Correlation")
plt.grid()
plt.show()

The plot reveals a strong correlation at lag 0, indicating that the two series change together simultaneously. As we increase the lag, the correlation slowly decreases but remains relatively high up to 14 time units, showing a positive relationship that weakens over time. In summary, the plot demonstrates that Series_4 and Series_3 are closely connected, and this relationship continues even when one series is shifted over time.

## 6.3 Dimensionality Reduction: Principal Component Analysis (PCA)

We create a PCA model and specify that we want to keep 5 principal components (PCs). We then transform the standardized data (discussed later in more detail) to extract these components, which capture the most important patterns in the data.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Standardize the data
scaler = StandardScaler()
standardized_data = scaler.fit_transform(data)

# calculate PCs
pca = PCA(n_components=5)  # Adjust number of components if necessary
pca_scores = pca.fit_transform(standardized_data)
pca_df = pd.DataFrame(data=pca_scores, columns=[f'PC_{i + 1}' for i in range(pca.n_components_)], index=data.index)

The explained variance in the PCA describes how much variance (or information) each PC captures. Higher values mean the component is more important. It helps us understand how many components we might need to keep for analysis.

In [ ]:
# Plotting the explained variance
plt.figure()
plt.plot(range(1, 6), pca.explained_variance_ratio_, marker='o')
plt.title('Explained Variance by Principal Components')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.grid()
plt.show()

We visualize the values of the first two PCs over time. This helps us see how the main patterns in the data change, making it easier to explore trends and relationships in the dataset.

In [ ]:
# Plotting the first two principal components

# time series plot
plt.figure()
plt.plot(pca_df.index, pca_df['PC_1'], label='Principal Component 1', color='blue')
plt.plot(pca_df.index, pca_df['PC_2'], label='Principal Component 2', color='blue', linestyle='--')
plt.title('First Two Principal Components')
plt.xlabel('Date')
plt.ylabel('Values')
plt.legend()
plt.grid()
plt.tight_layout()  # Adjust the layout
plt.show()

# scatterplot
plt.figure()
sns.scatterplot(data=pca_df, x='PC_1', y='PC_2', color='blue')
plt.title('First Two Principal Components')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid()
plt.tight_layout()
plt.show()